## O'Neal, Axl - Lab 1: Planning Observations

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pytz
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.table import QTable
from astroplan import Observer, FixedTarget
from astroplan import time_grid_from_range
from astroplan.plots import plot_sky, plot_airmass
from astroplan import download_IERS_A
from astroplan import moon_illumination
from astroplan import AtNightConstraint, MoonSeparationConstraint
from astroplan import AltitudeConstraint, AirmassConstraint
from astroplan import observability_table
download_IERS_A

import warnings
warnings.filterwarnings('ignore', category=Warning)

From Open Supernova Catalog I downloaded a csv of supernovae that includes Name, RA, and DEC

In [14]:
## Initialize list of supernovae
super_table = QTable.read('./supernovae.csv', format='ascii.csv')

## Trim RA column into usable format
for idx, val in enumerate(super_table['RA']):
    ra = super_table['RA'][idx].split(',',1)[0]
    ra_list = ra.split(':')
    ## RA to format 00h00m00.00s
    ra = ra_list[0]+'h'+ra_list[1]+'m'+ra_list[2]+'s'
    super_table['RA'][idx] = ra

## Trim Dec column into usable format
for idx, val in enumerate(super_table['DEC']):
    dec = super_table['DEC'][idx].split(',',1)[0]
    dec_list = dec.split(':')
    ## DEC to format +/-00d00m00.00s
    dec = dec_list[0]+'d'+dec_list[1]+'m'+dec_list[2]+'s'
    super_table['DEC'][idx] = dec

In [15]:
## Use list of targets to create a list of FixedTargets compatible with astroplan
supernovae = [FixedTarget(coord=SkyCoord(ra = RA, dec = DEC), name=Name)
             for Name, RA, DEC in super_table]

Observation site assigned: Subaru Observatory

Observation date assigned: November 1-15, 2018

In [16]:
## Initialize observation site
observer = Observer.at_site("Subaru", timezone="US/Hawaii")
## Initialize observation range of dates
date_start = Time("2018-11-01", format='iso')
date_end = Time("2018-11-15", format='iso')
observe_start = observer.sun_set_time(date_start, which='nearest')
observe_end = observer.sun_rise_time(date_end, which='next')
observing_range = [observe_start, observe_end]

time_grid = time_grid_from_range(observing_range)

In [19]:
## Counts the total number of objects up from site during observing range
n = 0
observables = []
for target in supernovae:
    if observer.target_is_up(observing_range, target).any():
        observables.append(target)
        n+=1
print(str(n) + ' out of the ' + str(len(supernovae)) 
      + ' objects are above the horizon at some point during 01-11-18 to 15-11-18')
observables

87 out of the 99 objects are above the horizon at some point during 01-11-18 to 15-11-18


[<FixedTarget "SN2011fe" at SkyCoord (ICRS): (ra, dec) in deg ( 210.77379583,  54.27367222)>,
 <FixedTarget "SN2003dh" at SkyCoord (ICRS): (ra, dec) in deg ( 161.20845833,  21.52170833)>,
 <FixedTarget "SN1993J" at SkyCoord (ICRS): (ra, dec) in deg ( 148.85322792,  69.02047278)>,
 <FixedTarget "SN2002ap" at SkyCoord (ICRS): (ra, dec) in deg ( 24.099375,  15.75366667)>,
 <FixedTarget "CSS141118:092034+504148" at SkyCoord (ICRS): (ra, dec) in deg ( 140.14275,  50.69677778)>,
 <FixedTarget "SN2009ip" at SkyCoord (ICRS): (ra, dec) in deg ( 335.78458333, -28.94777778)>,
 <FixedTarget "SN2011dh" at SkyCoord (ICRS): (ra, dec) in deg ( 202.52127292,  47.16970056)>,
 <FixedTarget "SN2012dn" at SkyCoord (ICRS): (ra, dec) in deg ( 305.90108333, -28.27872222)>,
 <FixedTarget "SN2013ej" at SkyCoord (ICRS): (ra, dec) in deg ( 24.20066667,  15.75861111)>,
 <FixedTarget "SN2012aw" at SkyCoord (ICRS): (ra, dec) in deg ( 160.97389583,  11.67156389)>,
 <FixedTarget "SN1999ee" at SkyCoord (ICRS): (ra, dec

In [20]:
## Set constraints to limit moon interference and account for airmass
constraints = [MoonSeparationConstraint(45*u.deg), AtNightConstraint(),
              AirmassConstraint(2)]
observing_table = observability_table(constraints, observer, supernovae, time_range=observing_range)

target name,ever observable,always observable,fraction of time observable
str23,bool,bool,float64
SN1987A,False,False,0.0
SN2011fe,False,False,0.0
SN2003dh,True,False,0.0859598853868
SN1993J,True,False,0.136103151862
SN2002ap,True,False,0.361031518625
CSS141118:092034+504148,True,False,0.164756446991
SN2009ip,True,False,0.159025787966
SN2011dh,True,False,0.0114613180516
SN2012dn,True,False,0.0744985673352


In [24]:
## Saves a csv file of observable supernovae in first half of Nov
observe_mask = np.where(observing_table['ever observable'])
super_final = super_table[observe_mask]
super_final.write('observable_supernovae.csv', format='ascii.csv')